In [1]:
# Importa bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
# Datos de entrenamiento crudos
df_entrenamiento = pd.read_csv('./data/raw/hoteles-entrena.csv')
df_entrenamiento.head()

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,market_segment,distribution_channel,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,average_daily_rate,required_car_parking_spaces,total_of_special_requests,arrival_date
0,Resort_Hotel,342,0,0,2,none,BB,PRT,Direct,Direct,...,3,No_Deposit,NaN,NaN,0,Transient,0.0,none,0,2015-07-01
1,Resort_Hotel,737,0,0,2,none,BB,PRT,Direct,Direct,...,4,No_Deposit,NaN,NaN,0,Transient,0.0,none,0,2015-07-01
2,Resort_Hotel,7,0,1,1,none,BB,GBR,Direct,Direct,...,0,No_Deposit,NaN,NaN,0,Transient,75.0,none,0,2015-07-01
3,Resort_Hotel,13,0,1,1,none,BB,GBR,Corporate,Corporate,...,0,No_Deposit,304.0,NaN,0,Transient,75.0,none,0,2015-07-01
4,Resort_Hotel,14,0,2,2,none,BB,GBR,Online_TA,TA/TO,...,0,No_Deposit,240.0,NaN,0,Transient,98.0,none,1,2015-07-01


In [3]:
# Prepara datos de entrenamiento
df_prueba = df_entrenamiento[['children', 'required_car_parking_spaces', 'hotel', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'country', 'booking_changes', 'customer_type', 'average_daily_rate', 'market_segment']]

df_prueba['children'] = df_prueba['children'].replace('none', 0)
df_prueba['children'] = df_prueba['children'].replace('children', 1)

df_prueba['required_car_parking_spaces'] = df_prueba['required_car_parking_spaces'].replace('none', 0)
df_prueba['required_car_parking_spaces'] = df_prueba['required_car_parking_spaces'].replace('parking', 1)

label_encoder = LabelEncoder()
df_prueba['hotel'] = label_encoder.fit_transform(df_prueba['hotel'])
df_prueba['country'] = label_encoder.fit_transform(df_prueba['country'])
df_prueba['customer_type'] = label_encoder.fit_transform(df_prueba['customer_type'])

df_prueba['average_daily_rate'] = df_prueba['average_daily_rate'] / 100

df_prueba['country'] = df_prueba['country'] / df_prueba['country'].max()

# Aplicar OneHotEncoding a la variable 'market_segment'
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoded = onehot_encoder.fit_transform(df_prueba[['market_segment']]).toarray()
column_names = onehot_encoder.get_feature_names_out(['market_segment'])
df_prueba[column_names] = onehot_encoded
df_prueba = df_prueba.drop(columns=['market_segment'])

df_prueba.head()

/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_43553/4231165155.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prueba['children'] = df_prueba['children'].replace('none', 0)
/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_43553/4231165155.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_prueba['children'] = df_prueba['children'].replace('children', 1)
/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_43553/4231165155.py:5: SettingWithCopyWarning: 
A value is trying to be set

,children,required_car_parking_spaces,hotel,stays_in_weekend_nights,stays_in_week_nights,adults,country,booking_changes,customer_type,average_daily_rate,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline_TA/TO,market_segment_Online_TA
0,0,0,1,0,0,2,0.780822,3,2,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,0,1,0,0,2,0.780822,4,2,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,1,0,1,1,0.356164,0,2,0.75,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,0,1,0,1,1,0.356164,0,2,0.75,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,0,1,0,2,2,0.356164,0,2,0.98,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
# Importa bibliotecas para entrenar modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_prueba.drop('children', axis=1)  # Variables predictoras
y = df_prueba['children']               # Variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Crear el modelo de clasificación
clf = RandomForestClassifier()

# Entrenar el modelo
clf.fit(X_train, y_train)

#Predice la probabilidad de que un cliente tenga hijos en escala de 0 a 1
y_pred_proba = clf.predict_proba(X_test)

# Hacer predicciones
y_pred = clf.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.92


In [6]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print(f'Matriz de Confusión:\n {cm}')

# Reporte de clasificación
report = classification_report(y_test, y_pred)
print(f'Reporte de Clasificación:\n {report}')

Matriz de Confusión:
 [[14326   259]
 [  977   333]]
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96     14585
           1       0.56      0.25      0.35      1310

    accuracy                           0.92     15895
   macro avg       0.75      0.62      0.65     15895
weighted avg       0.91      0.92      0.91     15895



### Predicción para datos hoteles-prueba.csv

In [7]:
# Preprocesa datos de prueba concurso
df_concurso = pd.read_csv('./data/raw/hoteles-prueba.csv')

# Prepara datos de entrenamiento
df_prueba_concurso = df_concurso[['id', 'required_car_parking_spaces', 'hotel', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'country', 'booking_changes', 'customer_type', 'average_daily_rate', 'market_segment']]

df_prueba_concurso['required_car_parking_spaces'] = df_prueba_concurso['required_car_parking_spaces'].replace('none', 0)
df_prueba_concurso['required_car_parking_spaces'] = df_prueba_concurso['required_car_parking_spaces'].replace('parking', 1)

label_encoder = LabelEncoder()
df_prueba_concurso['hotel'] = label_encoder.fit_transform(df_prueba_concurso['hotel'])
df_prueba_concurso['country'] = label_encoder.fit_transform(df_prueba_concurso['country'])
df_prueba_concurso['customer_type'] = label_encoder.fit_transform(df_prueba_concurso['customer_type'])

df_prueba_concurso['average_daily_rate'] = df_prueba_concurso['average_daily_rate'] / 100

df_prueba_concurso['country'] = df_prueba_concurso['country'] / df_prueba_concurso['country'].max()

# Aplicar OneHotEncoding a la variable 'market_segment'
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoded = onehot_encoder.fit_transform(df_prueba_concurso[['market_segment']]).toarray()
column_names = onehot_encoder.get_feature_names_out(['market_segment'])
df_prueba_concurso[column_names] = onehot_encoded
df_prueba_concurso = df_prueba_concurso.drop(columns=['market_segment'])

df_prueba_concurso.head()

/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_43553/2109944693.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prueba_concurso['required_car_parking_spaces'] = df_prueba_concurso['required_car_parking_spaces'].replace('none', 0)
/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_43553/2109944693.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_prueba_concurso['required_car_parking_spaces'] = df_prueba_concurso['required_car_parking_spaces'].replace('parking', 1)
/var/folders/mn/vp70xqd

,id,required_car_parking_spaces,hotel,stays_in_weekend_nights,stays_in_week_nights,adults,country,booking_changes,customer_type,average_daily_rate,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline_TA/TO,market_segment_Online_TA
0,1,0,1,0,2,2,0.340909,0,2,0.9800,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0,1,0,4,2,0.954545,0,2,0.9700,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0,1,0,4,2,0.295455,1,2,1.5477,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0,1,0,4,2,0.424242,0,2,0.9700,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,1,0,4,2,0.750000,0,0,0.9750,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
X_concurso = df_prueba_concurso.drop('id', axis=1)
id = df_prueba_concurso['id']

y_pred_proba_concurso = clf.predict_proba(X_concurso)

#Crea un dataframe con id y probabilidad de que un cliente tenga hijos
df_concurso_res = pd.DataFrame({'id': id, 'children': y_pred_proba_concurso[:,1]})

# Guardar el dataframe en un archivo CSV
df_concurso_res.to_csv('./data/submission_res.csv', index=False)